In [ ]:
import random
from copy import deepcopy

BlankSpace = 0
StartState = []
GoalState = [[1,2,3],[4,5,6],[7,8,BlankSpace]]

def RandomizeState():
    PossibleValues = [1,2,3,4,5,6,7,8,BlankSpace]
    RandomizedList = []

    for _ in range(3):
        NewRow = []

        for _ in range(3):
            RandomElementPosition = random.randint(0,len(PossibleValues)-1)
            NewRow.append(PossibleValues.pop(RandomElementPosition))
        
        RandomizedList.append(NewRow)
    
    return RandomizedList

def GetPositionFromState(state,num):
    IndexRow = IndexCol = -1
    i = j = 0
    FoundNum = False

    while i < 3 and not FoundNum:
        while j < 3 and not FoundNum:
            if state[i][j] == num:
                IndexRow = i
                IndexCol = j
            j+=1
        j=0
        i+=1
    
    return IndexRow,IndexCol

def PrintPosition(state):
    for i in range(3): 
        for j in range(3): 
            print(f" {state[i][j]} ",end="")
        print("")

def IsSolvable(state):
    flat = [cell for row in state for cell in row if cell != 0]
        
    inversions = 0
    for i in range(len(flat)):
        for j in range(i + 1, len(flat)):
            if flat[i] > flat[j]:
                inversions += 1
    
    return inversions % 2 == 0

StartState = RandomizeState()
while not IsSolvable(StartState):
    StartState = RandomizeState()
    
print("Stato iniziale:\n")
PrintPosition(StartState)

In [ ]:
# Muovere un tassello può essere visto come muovere lo spazio vuoto;
def ExpandNodes(CurrentState):
    Queue = []
    IndexRow = IndexCol = -1
    i = j = 0
    FoundBlank = False

    while i < 3 and not FoundBlank:
        while j < 3 and not FoundBlank:
            if CurrentState[i][j] == BlankSpace:
                FoundBlank = True
                IndexRow = i
                IndexCol = j
            j+=1
        j=0
        i+=1

    if IndexCol+1 < 3:
        MoveRight = deepcopy(CurrentState)

        MoveRight[IndexRow][IndexCol] = MoveRight[IndexRow][IndexCol+1]
        MoveRight[IndexRow][IndexCol+1] = BlankSpace
        Queue.append(MoveRight)

    if IndexCol-1 >= 0:
        MoveLeft = deepcopy(CurrentState)

        MoveLeft[IndexRow][IndexCol] = MoveLeft[IndexRow][IndexCol-1]
        MoveLeft[IndexRow][IndexCol-1] = BlankSpace
        Queue.append(MoveLeft)

    if IndexRow+1 < 3:
        MoveUp = deepcopy(CurrentState)

        MoveUp[IndexRow][IndexCol] = MoveUp[IndexRow+1][IndexCol]
        MoveUp[IndexRow+1][IndexCol] = BlankSpace
        Queue.append(MoveUp)
    
    if IndexRow-1 >= 0:
        MoveDown = deepcopy(CurrentState)

        MoveDown[IndexRow][IndexCol] = MoveDown[IndexRow-1][IndexCol]
        MoveDown[IndexRow-1][IndexCol] = BlankSpace
        Queue.append(MoveDown)
    
    return Queue

def SearchLowest(StateList):
    lowest = float("inf")
    LowestNodeIndex = -1
    i = 0

    while i < len(StateList):

        if StateList[i][1] < lowest:
            lowest = StateList[i][1]
            LowestNodeIndex = i
        
        i+=1

    return LowestNodeIndex

In [ ]:
def SolveRelaxedProblem(CurrentState):
    OrderedPosition = [1,2,3,4,5,6,7,8,0]
    MismatchedNumbers = 0

    for i in range(3):
        for j in range(3):
            if CurrentState[i][j] == BlankSpace: continue
            if CurrentState[i][j] != OrderedPosition[(i*3)+j]:
                MismatchedNumbers += 1

    return MismatchedNumbers

def A_star(StartState):
    queue = []
    visited = []
    CurrentStep = 0

    queue.append((StartState,SolveRelaxedProblem(StartState),0))

    while len(queue) != 0:
        index = SearchLowest(queue)
        CurrentNode = queue.pop(index)

        if CurrentNode[0] in visited: continue
        visited.append(CurrentNode[0])

        print(f"Step n.{CurrentStep}, posizione attuale:")
        PrintPosition(CurrentNode[0])
        print(f"Valutazione della posizione: {SolveRelaxedProblem(CurrentNode[0])}")
        print(f"f(n) = {CurrentNode[1]}")
        print(f"Depth: {CurrentNode[2]}\n")

        if CurrentNode[0] == GoalState:
            return CurrentNode
        
        NewNodes = ExpandNodes(CurrentNode[0])
        for Node in NewNodes:
            g = CurrentNode[2]+1
            f = g + SolveRelaxedProblem(Node)

            queue.append((Node,f,g))
        
        CurrentStep += 1

    return "failure"

print(A_star(StartState))

In [ ]:
def SolveRelaxedProblem(CurrentState):
    GridCharacter = [1,2,3,4,5,6,7,8]
    ManhattanDist = 0

    for num in GridCharacter:
        i,j = GetPositionFromState(GoalState,num)
        k,h = GetPositionFromState(CurrentState,num)
        ManhattanDist += (abs(i-k)+abs(j-h))
    
    return ManhattanDist

def A_star(StartState):
    queue = []
    visited = []
    CurrentStep = 0

    queue.append((StartState,SolveRelaxedProblem(StartState),0))

    while len(queue) != 0:
        index = SearchLowest(queue)
        CurrentNode = queue.pop(index)

        if CurrentNode[0] in visited: continue
        visited.append(CurrentNode[0])

        print(f"Step n.{CurrentStep}, posizione attuale:")
        PrintPosition(CurrentNode[0])
        print(f"Valutazione della posizione: {SolveRelaxedProblem(CurrentNode[0])}")
        print(f"f(n) = {CurrentNode[1]}")
        print(f"Depth: {CurrentNode[2]}\n")

        if CurrentNode[0] == GoalState:
            return CurrentNode
        
        NewNodes = ExpandNodes(CurrentNode[0])
        for Node in NewNodes:
            g = CurrentNode[2]+1
            f = g + SolveRelaxedProblem(Node)
            queue.append((Node,f,g))
        
        CurrentStep += 1

    return "failure"

print(A_star(StartState))